In [1]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql.functions import *
spark=SparkSession \
        .builder \
        .appName("Bollywood over 3 rating")\
        .master("spark://ubuntu:7077")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [3]:
stagemetrics.begin()
Ratings_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/rating.csv")
      )
Tags_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/tag.csv")
      )
Tags_Dataframe=Tags_Dataframe.withColumn('tag',lower(col('tag'))).withColumn('tag',regexp_replace('tag', '[^\w\s]+', ''))
Joined_Dataframe=(Tags_Dataframe.filter(Tags_Dataframe.tag=='bollywood') 
                                .join(Ratings_Dataframe,[(Ratings_Dataframe.userId==Tags_Dataframe.userId)& (Ratings_Dataframe.movieId==Tags_Dataframe.movieId)])
                                .where(Ratings_Dataframe.rating>3)
                                .dropDuplicates(['userId'])
                                .select(Tags_Dataframe.userId,Tags_Dataframe.movieId,Tags_Dataframe.tag,Ratings_Dataframe.rating)
                 )
Joined_Dataframe.sort("userId").show()
stagemetrics.end()

+------+-------+---------+------+
|userId|movieId|      tag|rating|
+------+-------+---------+------+
| 10573|   6776|bollywood|   4.0|
| 19837| 108514|bollywood|   5.0|
| 23333|  63082|bollywood|   4.0|
| 25004|  63082|bollywood|   5.0|
| 31338|   6776|bollywood|   4.5|
| 33323|  78408|bollywood|   3.5|
| 35170|  91241|bollywood|   4.0|
| 40514|   6776|bollywood|   5.0|
| 41165|  63082|bollywood|   4.5|
| 51539|  63082|bollywood|   4.0|
| 54900|  97648|bollywood|   3.5|
| 63618|  56167|bollywood|   3.5|
| 65908|   6776|bollywood|   4.5|
| 70279|   5135|bollywood|   4.5|
| 77137|  55098|bollywood|   5.0|
| 86883|   5135|bollywood|   4.0|
|124139|  74641|bollywood|   3.5|
|131829|   6776|bollywood|   4.0|
|135155|   5135|bollywood|   4.5|
|136015|  63082|bollywood|   4.5|
+------+-------+---------+------+
only showing top 20 rows



In [4]:
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 4
Aggregated Spark stage metrics:
numStages => 8
numTasks => 422
elapsedTime => 67535 (1.1 min)
stageDuration => 68918 (1.1 min)
executorRunTime => 203084 (3.4 min)
executorCpuTime => 78199 (1.3 min)
executorDeserializeTime => 15014 (15 s)
executorDeserializeCpuTime => 3676 (4 s)
resultSerializationTime => 473 (0.5 s)
jvmGCTime => 2912 (3 s)
shuffleFetchWaitTime => 9 (9 ms)
shuffleWriteTime => 1854 (2 s)
resultSize => 1606858 (1569.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 2219835392
recordsRead => 33126961
bytesRead => 1425386090 (1359.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 6221891
shuffleTotalBlocksFetched => 1054
shuffleLocalBlocksFetched => 1054
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 94023746 (89.0 MB)
shuffleLocalBytesRead => 94023746 (89.0 MB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadTo